In [11]:
!pip install z3-solver


In [14]:

from z3 import BitVec, Solver, And, Or, Not

# Define 8-bit symbolic bit-vectors
x = BitVec('x', 8)  # 8-bit integer
y = BitVec('y', 8)  # Another 8-bit integer

# Create a solver instance
solver = Solver()

# Add bitwise constraints
solver.add(x & y == 0x3)       # Bitwise AND constraint
solver.add(x | y == 0x7)       # Bitwise OR constraint
solver.add(x ^ y == 0x4)       # Bitwise XOR constraint
solver.add(Not(x == 0))        # Ensure x is not zero

# Add additional constraints if needed
solver.add(x << 1 == y)        # Bitwise left shift of x by 1 should equal y

# Check satisfiability
if solver.check() == z3.sat:
    model = solver.model()
    print("Solution found:")
    print(f"x = {model[x].as_long():02x}")
    print(f"y = {model[y].as_long():02x}")
else:
    print("No solution satisfies the constraints.")


ImportError: cannot import name 'BitVec' from 'z3' (/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/z3/__init__.py)

In [21]:
from cvc5 import Solver, Kind

# Initialize cvc5 solver
solver = Solver()
solver.setOption("produce-models", "true")
solver.setLogic("QF_BV")  # Quantifier-free bit-vector logic for fixed-width fields

# Define TCP Header Fields as symbolic bit-vectors
# 16-bit source port, 16-bit destination port, 8-bit flags
source_port = solver.mkConst(solver.mkBitVectorSort(16), "source_port")
dest_port = solver.mkConst(solver.mkBitVectorSort(16), "dest_port")
flags = solver.mkConst(solver.mkBitVectorSort(8), "flags")

# Define flag bit values
URG_FLAG = solver.mkBitVector(8, 0x20)  # URG flag (0b00100000)
ACK_FLAG = solver.mkBitVector(8, 0x10)  # ACK flag (0b00010000)
PSH_FLAG = solver.mkBitVector(8, 0x08)  # PSH flag (0b00001000)
RST_FLAG = solver.mkBitVector(8, 0x04)  # RST flag (0b00000100)
SYN_FLAG = solver.mkBitVector(8, 0x02)  # SYN flag (0b00000010)
FIN_FLAG = solver.mkBitVector(8, 0x01)  # FIN flag (0b00000001)

# Example combinations of flags
# SYN-ACK Packet (e.g., for connection establishment)
syn_ack_packet = solver.mkTerm(Kind.EQUAL, flags, solver.mkBitVector(8, 0x12))  # 0b00010010

# FIN-ACK Packet (e.g., for connection termination)
fin_ack_packet = solver.mkTerm(Kind.EQUAL, flags, solver.mkBitVector(8, 0x11))  # 0b00010001

# Add constraints to the solver
# Here, we can try different scenarios by asserting different flag combinations.
# Let's first test for a SYN-ACK packet to destination port 80
is_dest_port_80 = solver.mkTerm(Kind.EQUAL, dest_port, solver.mkBitVector(16, 80))
syn_ack_constraints = solver.mkTerm(Kind.AND, syn_ack_packet, is_dest_port_80)

# Assert the constraints for a SYN-ACK packet
solver.assertFormula(syn_ack_constraints)

# Check if there's a valid assignment (satisfiable model) for the SYN-ACK constraints
if solver.checkSat().isSat():
    # Get model and display values for source_port, dest_port, and flags
    model_source_port = solver.getValue(source_port)
    model_dest_port = solver.getValue(dest_port)
    model_flags = solver.getValue(flags)

    print("SYN-ACK Packet Model:")
    print("Source Port:", str(model_source_port))
    print("Destination Port:", str(model_dest_port))
    print("Flags:", str(model_flags))
else:
    print("No satisfiable solution found for the SYN-ACK constraints.")

# Reset solver for another check
solver.resetAssertions()

# Assert the constraints for a FIN-ACK packet to the same destination port 80
fin_ack_constraints = solver.mkTerm(Kind.AND, fin_ack_packet, is_dest_port_80)
solver.assertFormula(fin_ack_constraints)

# Check if there's a valid assignment (satisfiable model) for the FIN-ACK constraints
if solver.checkSat().isSat():
    # Get model and display values for source_port, dest_port, and flags
    model_source_port = solver.getValue(source_port)
    model_dest_port = solver.getValue(dest_port)
    model_flags = solver.getValue(flags)

    print("FIN-ACK Packet Model:")
    print("Source Port:", str(model_source_port))
    print("Destination Port:", str(model_dest_port))
    print("Flags:", str(model_flags))
else:
    print("No satisfiable solution found for the FIN-ACK constraints.")


SYN-ACK Packet Model:
Source Port: #b0000000000000000
Destination Port: #b0000000001010000
Flags: #b00010010
FIN-ACK Packet Model:
Source Port: #b0000000000000000
Destination Port: #b0000000001010000
Flags: #b00010001


In [13]:
from z3 import Extract

# Extract the lower 4 bits of an 8-bit bit-vector
low_bits = Extract(3, 0, x)  # Extract bits 3 through 0 of x
solver.add(low_bits == 0b0011)


ImportError: cannot import name 'Extract' from 'z3' (/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/z3/__init__.py)

In [ ]:
import angr
import claripy

# Define a binary to analyze
binary_path = '/path/to/binary'
project = angr.Project(binary_path, auto_load_libs=False)

# Define symbolic bytes as input
symbolic_input = claripy.BVS('input', 8 * 4)  # 4 bytes

# Run symbolic execution
state = project.factory.full_init_state(stdin=symbolic_input)
simgr = project.factory.simgr(state)
simgr.explore(find=0x400123)  # Example address to find

# Check for solutions
if simgr.found:
    solution = simgr.found[0].solver.eval(symbolic_input, cast_to=bytes)
    print(f'Solution found: {solution}')


In [ ]:
##ПОшукати байтови SOLVER - bitwise SOLVER byteWISE

In [7]:
from sympy import symbols, Eq, solve

# Define variables
T = symbols('T')  # Temperature at a given time
T0 = symbols('T0')  # Initial temperature
H = symbols('H')  # Heating or cooling factor
t = symbols('t')  # Time

# Define temperature equation: T = T0 + H * t
temperature_eq = Eq(T, T0 + H * t)

# Solve for T in terms of T0, H, and t
temperature_formula = solve(temperature_eq, T)[0]
print(f"Temperature formula based on heating/cooling: {temperature_formula}")

# Generate a sequence of temperatures based on initial conditions for LSTM input
initial_temp = 20  # Starting temperature
heating_factor = 1.5  # Heating factor per time unit
time_series = [(i, float(temperature_formula.subs({T0: initial_temp, H: heating_factor, t: i}))) for i in range(10)]
print("Generated temperature time series:", time_series)


Temperature formula based on heating/cooling: H*t + T0
Generated temperature time series: [(0, 20.0), (1, 21.5), (2, 23.0), (3, 24.5), (4, 26.0), (5, 27.5), (6, 29.0), (7, 30.5), (8, 32.0), (9, 33.5)]


In [5]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Convert time series to numpy array for LSTM
data = np.array([value for _, value in time_series])
data = data.reshape((-1, 1))

# Define a sequence length and generate time-series batches for LSTM
sequence_length = 3
generator = TimeseriesGenerator(data, data, length=sequence_length, batch_size=1)


ImportError: dlopen(/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/tensorflow/python/platform/_pywrap_cpu_feature_guard.so, 0x0002): Library not loaded: @rpath/libtensorflow_cc.2.dylib
  Referenced from: <327AFC74-CE12-3903-8CAA-371285A02F18> /Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so
  Reason: tried: '/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/tensorflow/python/../../_solib_darwin_x86_64/libtensorflow_cc.2.dylib' (no such file), '/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so.runfiles/org_tensorflow/_solib_darwin_x86_64/libtensorflow_cc.2.dylib' (no such file), '/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/tensorflow/python/../../_solib_darwin_x86_64/_U_S_Stensorflow_Spython_C_Upywrap_Utensorflow_Uinternal.so_Ucclib___Utensorflow/libtensorflow_cc.2.dylib' (no such file), '/Users/smander/pro

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the LSTM model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(sequence_length, 1)),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(generator, epochs=100)


In [ ]:
# Predict the next value
predicted_temperature = model.predict(generator)

# Apply constraints post-prediction using symbolic AI
# For instance, ensure the predicted temperature does not exceed physical limits
max_temp = 35  # Maximum allowed temperature (constraint)
adjusted_prediction = min(predicted_temperature[0][0], max_temp)

print(f"Predicted temperature: {predicted_temperature[0][0]}, Adjusted for constraint: {adjusted_prediction}")



In [2]:
!pip install --upgrade pip
!pip install symbolic-ai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
ERROR: Could not find a version that satisfies the requirement symbolic-ai (from versions: none)
ERROR: No matching distribution found for symbolic-ai


In [3]:
!pip install symai
import symai as ai

ERROR: Could not find a version that satisfies the requirement symai (from versions: none)
ERROR: No matching distribution found for symai


ModuleNotFoundError: No module named 'symai'

In [ ]:
from symbolic import Symbol, Rule, KnowledgeBase

# Define symbols (variables) for packet attributes
packet_size = Symbol('packet_size')
is_large_packet = Symbol('is_large_packet')

# Define rules for the knowledge base
rules = [
    Rule(packet_size >= 64, "Packet size must be at least 64 bytes."),
    Rule(packet_size <= 1500, "Packet size must not exceed 1500 bytes."),
    Rule(packet_size > 1000, "Large Packet", implies=is_large_packet)
]

# Create a knowledge base and add rules
kb = KnowledgeBase(rules)

# Check if a given packet size satisfies all constraints
packet_example = {'packet_size': 2000}  # Hypothetical packet size

if kb.satisfies(packet_example):
    print("Packet satisfies protocol constraints.")
else:
    print("Packet violates protocol constraints.")


In [ ]:
# Define a packet size that we want to check
packet_example = {'packet_size': 1200}

# Add inference capability to deduce if the packet is large
inferred = kb.infer(packet_example)
print(f"Inferred properties: {inferred}")


In [ ]:
# Predicted packet size from LSTM
predicted_size = 1800  # Example neural network prediction

# Validate prediction against symbolic constraints
packet_example = {'packet_size': predicted_size}
if kb.satisfies(packet_example):
    print(f"Predicted size {predicted_size} is valid.")
else:
    # Adjust the prediction to fit within constraints
    adjusted_size = min(max(predicted_size, 64), 1500)
    print(f"Adjusted size to meet constraints: {adjusted_size}")
